In [22]:
# General:
import tweepy           # To consume Twitter's API
import pandas as pd     # To handle data
import numpy as np      # For number computing
import json

# For plotting and visualization:
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [23]:
# Twitter App access keys for @user

# Consume:
CONSUMER_KEY = "nui9d52bUoGa1tZBvvy4HGxTl"
CONSUMER_SECRET = "qTJ5W8EX2b2eFvKtSw80wIw714Hs9rXpUsb0RVwiGxAbyS4cnl"

# Access:
ACCESS_TOKEN = "1034084192936763393-FFXRmlq3PaoBWjjCdA7GLeP9kEIJD8"
ACCESS_SECRET = "tQvaBmXK0qlS2GSaxB7cLWneAVieYeomxN3ED8KrOp1b1"


# API's setup:
def twitter_setup():

    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication:
    api = tweepy.API(auth)
    return api

In [24]:
# We create an extractor object:
extractor = twitter_setup()

# We create a tweet list as follows:
tweets = extractor.user_timeline(screen_name="elonmusk", count=200)
print("Number of tweets extracted: {}.\n".format(len(tweets)))

# We print the most recent 5 tweets:
print("5 recent tweets:\n")
for tweet in tweets[:5]:
    print(tweet.text)
    print()


Number of tweets extracted: 200.

5 recent tweets:

@joerogan Glad you like it

RT @cleantechnica: Joe Rogan On Tesla Model S: “The Most Preposterously Quick Vehicle I’ve Ever Driven” https://t.co/DnV1cMaEYr https://t.c…

Tornado warning app is amazing except for this one little flaw … https://t.co/L03Gjz8IQY https://t.co/YxebWu5eFv

@psalman03 @Erdayastronaut @dguisinger @DanielDavisA Spark plugs ignite dual blow torches that ignite preburners &amp; main chamber

@Erdayastronaut @dguisinger @DanielDavisA Gaseous CH4/O2 &amp; heavy duty spark plugs. Basically, a 💨 of insane power 😀



In [25]:
# We create a pandas dataframe as follows:
data = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

# We display the first 10 elements of the dataframe:
display(data.head(10))

,Tweets
0,@joerogan Glad you like it
1,RT @cleantechnica: Joe Rogan On Tesla Model S:...
2,Tornado warning app is amazing except for this...
3,@psalman03 @Erdayastronaut @dguisinger @Daniel...
4,@Erdayastronaut @dguisinger @DanielDavisA Gase...
5,@jclishman @dguisinger @Erdayastronaut @Daniel...
6,@dguisinger @Erdayastronaut @DanielDavisA Engi...
7,https://t.co/yFhNx3fiOD
8,https://t.co/ChEGbjTGKZ
9,First firing of Starship Raptor flight engine!...


In [26]:
# We add relevant data:
data['len']  = np.array([len(tweet.text) for tweet in tweets])
data['ID']   = np.array([tweet.id for tweet in tweets])
data['Date'] = np.array([tweet.created_at for tweet in tweets])
data['Source'] = np.array([tweet.source for tweet in tweets])
data['Likes']  = np.array([tweet.favorite_count for tweet in tweets])
data['RTs']    = np.array([tweet.retweet_count for tweet in tweets])

# Display of first 10 elements from dataframe:
display(data.head(10))

,Tweets,len,ID,Date,Source,Likes,RTs
0,@joerogan Glad you like it,26,1092690014197014528,2019-02-05 07:42:55,Twitter for iPhone,15067,307
1,RT @cleantechnica: Joe Rogan On Tesla Model S:...,140,1092689678296203265,2019-02-05 07:41:35,Twitter for iPhone,0,292
2,Tornado warning app is amazing except for this...,112,1092678154877992960,2019-02-05 06:55:47,Twitter for iPhone,17600,1639
3,@psalman03 @Erdayastronaut @dguisinger @Daniel...,131,1092320321229643776,2019-02-04 07:13:53,Twitter for iPhone,1063,42
4,@Erdayastronaut @dguisinger @DanielDavisA Gase...,119,1092282107639021569,2019-02-04 04:42:02,Twitter for iPhone,2328,118
5,@jclishman @dguisinger @Erdayastronaut @Daniel...,139,1092281374436978689,2019-02-04 04:39:07,Twitter for iPhone,1268,47
6,@dguisinger @Erdayastronaut @DanielDavisA Engi...,140,1092280273599979520,2019-02-04 04:34:45,Twitter for iPhone,1937,75
7,https://t.co/yFhNx3fiOD,23,1092272377889738753,2019-02-04 04:03:22,Twitter for iPhone,76162,9464
8,https://t.co/ChEGbjTGKZ,23,1092270756715737088,2019-02-04 03:56:56,Twitter for iPhone,103772,14450
9,First firing of Starship Raptor flight engine!...,111,1092268892339273730,2019-02-04 03:49:31,Twitter for iPhone,139590,16373


In [27]:
from textblob import TextBlob
import re

def clean_tweet(tweet):
    '''
    Utility function to clean the text in a tweet by removing 
    links and special characters using regex.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def analize_sentiment(tweet):
    '''
    Utility function to classify the polarity of a tweet
    using textblob.
    '''
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity == 0:
        return 0
    else:
        return -1

In [28]:
# We create a column with the result of the analysis:
data['SA'] = np.array([ analize_sentiment(tweet) for tweet in data['Tweets'] ])

# We display the updated dataframe with the new column:
display(data.head(10))

,Tweets,len,ID,Date,Source,Likes,RTs,SA
0,@joerogan Glad you like it,26,1092690014197014528,2019-02-05 07:42:55,Twitter for iPhone,15067,307,1
1,RT @cleantechnica: Joe Rogan On Tesla Model S:...,140,1092689678296203265,2019-02-05 07:41:35,Twitter for iPhone,0,292,1
2,Tornado warning app is amazing except for this...,112,1092678154877992960,2019-02-05 06:55:47,Twitter for iPhone,17600,1639,1
3,@psalman03 @Erdayastronaut @dguisinger @Daniel...,131,1092320321229643776,2019-02-04 07:13:53,Twitter for iPhone,1063,42,1
4,@Erdayastronaut @dguisinger @DanielDavisA Gase...,119,1092282107639021569,2019-02-04 04:42:02,Twitter for iPhone,2328,118,-1
5,@jclishman @dguisinger @Erdayastronaut @Daniel...,139,1092281374436978689,2019-02-04 04:39:07,Twitter for iPhone,1268,47,1
6,@dguisinger @Erdayastronaut @DanielDavisA Engi...,140,1092280273599979520,2019-02-04 04:34:45,Twitter for iPhone,1937,75,-1
7,https://t.co/yFhNx3fiOD,23,1092272377889738753,2019-02-04 04:03:22,Twitter for iPhone,76162,9464,0
8,https://t.co/ChEGbjTGKZ,23,1092270756715737088,2019-02-04 03:56:56,Twitter for iPhone,103772,14450,0
9,First firing of Starship Raptor flight engine!...,111,1092268892339273730,2019-02-04 03:49:31,Twitter for iPhone,139590,16373,1


In [29]:
# We construct lists with classified tweets:

pos_tweets = [ tweet for index, tweet in enumerate(data['Tweets']) if data['SA'][index] > 0]
neu_tweets = [ tweet for index, tweet in enumerate(data['Tweets']) if data['SA'][index] == 0]
neg_tweets = [ tweet for index, tweet in enumerate(data['Tweets']) if data['SA'][index] < 0]

# We print percentages:

print("Percentage of positive tweets: {}%".format(len(pos_tweets)*100/len(data['Tweets'])))
print("Percentage of neutral tweets: {}%".format(len(neu_tweets)*100/len(data['Tweets'])))
print("Percentage de negative tweets: {}%".format(len(neg_tweets)*100/len(data['Tweets'])))

Percentage of positive tweets: 44.0%
Percentage of neutral tweets: 46.5%
Percentage de negative tweets: 9.5%
